In [ ]:
import sys
sys.path.append('REPLACE with dbfold_script_generator path') # Add parent directory to path for local simulation and utils modules
import simulation.simulation as sim
import os

In [ ]:
date = '202506' # Date is used to differentiate between different simulations of same protein
pdbroot = '1uao' # PDB filename without .pdb extension, this will be used as an identifier for the protein during the simulation
identifier = 'equil0' # This is to differentiate between different simulations of same protein; usually minimization, equilibration, and checkpoints
simulation_path = f'/REPLACE with scratch folder path/{date}_{pdbroot}/' # This will be the main simulation directory, recommended to use scratch folder
sim_generator = sim.Simulation(
    pdbroot,
    identifier, 
    simulation_path, # Main simultion directory (recommend to use scratch folder)
    '/REPLACE with MCPU directory path', # Path to MCPU directory
    umbrella=True, # Set to True if you want to use umbrella sampling
    temperature_replica=True, # Set to True if you want to use temperature replica
    constraint=False, # Set to True if you want to use constraints
)

Umbrella sampling is enabled.
Temperature replica exchange is enabled.


In [3]:
sim_generator.set_starting_structure(
    f'{os.getcwd()}/pdb/{pdbroot}.pdb'
) # This should be the absolute path to the PDB file of the starting structure
sim_generator.set_native_structure(
    f'{os.getcwd()}/pdb/{pdbroot}.pdb'
) # This should be the absolute path to the PDB file of the native structure
native_contacts = sim_generator.count_contacts_rounded(6)

sim_generator.set_umbrella_sampling(
    umbrella_max = native_contacts,
    replica_steps = 10000,
    umbrella_bias=0.02,
    umbrella_spacing = 10
)
sim_generator.set_temperature_replica(0.4, 0.025, 25, 10000)

sim_generator.set_montecarlo_parameters(
    mc_steps = 1000000000,
    log_steps = 1000000
)
sim_generator.set_slurm_resources(
    job_name=pdbroot,
    partition='shared',
    time='3-00:00',
    memory='144G',
    cpu_per_node=112,
    email='REPLACE_WITH_YOUR_EMAIL', # Replace with your email to receive notifications
) # Modify slurm resources as needed

Maximum tempareture will be 1.0


/net/h-nfs15-p/data/shakhnovich_lab/share/users/kibumpark/envs/dbfoldtools/lib/python3.10/site-packages/mdtraj/formats/pdb/pdbfile.py:208: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn(
/n/home01/kibumpark/group_folder/pkg/dbfold_script_generator/simulation/simulation.py:105: UserWarning: Temperature replica exchange is enabled. The number of replica steps and max exchange pairs will be set in the set_temperature_replica method.
  warnings.warn('Temperature replica exchange is enabled. The number of replica steps and max exchange pairs will be set in the set_temperature_replica method.')


In [4]:
sim_generator.compute_protein_parameters()

Protein parameter files already exist. Skipping computation. If you want to recompute, please set recompute=True


0

In [5]:
sim_generator.generate_config(f'{simulation_path}/{identifier}/cfg')

In [6]:
sim_generator.generate_submission_script(f'{simulation_path}/{identifier}/MCPU.sh', 'src_mpi_umbrella/fold_potential_mpi')

0

In [7]:
sim_generator.print_config()

!!
!! NATIVE PROTEIN DATA
!!
NATIVE_FILE                     /net/h-nfs15-p/data/shakhnovich_lab/share/users/kibumpark/pkg/dbfold_script_generator/notebooks/pdb/1uao.pdb
STRUCTURE_FILE                  /net/h-nfs15-p/data/shakhnovich_lab/share/users/kibumpark/pkg/dbfold_script_generator/notebooks/pdb/1uao.pdb
NATIVE_DIRECTORY                None
TEMPLATE_FILE                   /n/home01/kibumpark/scratch_folder/MCPU/202506_1uao/nothing.template
ALIGNMENT_FILE                  1lfb
PDB_OUT_FILE                    /n/home01/kibumpark/scratch_folder/MCPU/202506_1uao/equil0/MCPU_run/1uao
PROTEIN_NAME                    p1afvA
!!
!!
!! POTENTIAL PARAMETERS
!!
NO_NEW_CLASHES                  1
READ_POTENTIAL                  1Lfb
USE_GO_POTENTIAL                0
CLASH_WEIGHT                    0
RMS_WEIGHT                      0
HYDROGEN_BOND                   -2
NATIVE_ATTRACTION               0
NON_NATIVE_REPULSION            0
NON_SPECIFIC_ENERGY             0
!!
!! CONTACT DEFINITION
!!

0

In [8]:
sim_generator.print_slurm()

#!/bin/bash
#SBATCH -N 1
#SBATCH -c 1
#SBATCH -n 50
#SBATCH -p sapphire
#SBATCH -t 3-00:00
#SBATCH --mem=900G
#SBATCH -J 1uao
#SBATCH -o 1uao_%j.out
#SBATCH -e 1uao_%j.err
#SBATCH --mail-type=ALL
#SBATCH --mail-user=REPLACE_WITH_YOUR_EMAIL
# This file can be anywhere, but the dir that contains the cfg file
# must also have fold_potential_mpi. Then, the config_files dir should
# be one level up from that one.

module purge
module load gcc/12.2.0-fasrc01 openmpi/4.1.4-fasrc01

cd /n/home01/kibumpark/group_folder/pkg/dbfold/MCPU/src_mpi_umbrella/
srun -n ${SLURM_NTASKS} --mpi=pmi2 /n/home01/kibumpark/group_folder/pkg/dbfold/MCPU/src_mpi_umbrella/fold_potential_mpi /n/home01/kibumpark/scratch_folder/MCPU/202506_1uao//equil0/cfg


0

In [ ]:
sim_generator.submit_slurm()